In [28]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import json
import pm4py
from log_distance_measures.config import EventLogIDs
from log_distance_measures.control_flow_log_distance import control_flow_log_distance
from log_distance_measures.circadian_event_distribution import circadian_event_distribution_distance
from log_distance_measures.config import AbsoluteTimestampType, DEFAULT_CSV_IDS, discretize_to_hour
from log_distance_measures.relative_event_distribution import relative_event_distribution_distance
from log_distance_measures.n_gram_distribution import n_gram_distribution_distance
from log_distance_measures.config import AbsoluteTimestampType, DEFAULT_CSV_IDS, discretize_to_hour
from log_distance_measures.relative_event_distribution import relative_event_distribution_distance
from log_distance_measures.circadian_event_distribution import circadian_event_distribution_distance
from log_distance_measures.config import AbsoluteTimestampType, DEFAULT_CSV_IDS
from log_distance_measures.absolute_event_distribution import absolute_event_distribution_distance
from log_distance_measures.case_arrival_distribution import case_arrival_distribution_distance
from log_distance_measures.cycle_time_distribution import cycle_time_distribution_distance
import glob
from log_distance_measures.config import DistanceMetric
from log_distance_measures.earth_movers_distance import earth_movers_distance


In [24]:
from pm4py.objects.log import log as lg

def prefix_to_log(df, max_prefix):
    new_log = lg.EventLog()
    dic = dict()
    for index, row in df.iterrows():
        case_id = row['trace_id']
        new_case = lg.Trace({'caseid': case_id})
        for i in range(1, max_prefix):
            prefix = 'prefix_' + str(i)
            event = lg.Event()
            event['concept_name'] = row[prefix]
            event['start:timestamp'] = row['start:timestamp_'+str(i)]
            event['time:timestamp'] =  row['time:timestamp_'+str(i)]
            new_case.insert(i, event)
        new_log.append(new_case)
    return new_log

In [41]:
original_train = pd.read_csv("/Users/francescameneghello/Documents/GitHub/nirdizati-light/experiments/new_logs/" +
                             "Productions_train_df_cf_aug_0.1_pref_len_20.csv")
#original_train = prefix_to_log(original_train, 9)

In [48]:
list(original_train[original_train.trace_id == 'Case178']['prefix_2'])

['Round Grinding']

In [53]:
def transform_df(df, max_prefix):
    columns = ["trace_id", "task", "start_timestamp", "time_timestamp"]
    df_new = pd.DataFrame(columns=columns)
    for index, row in df.iterrows():
        case_id = row['trace_id']
        print(case_id)
        for i in range(1, max_prefix):
            prefix = 'prefix_' + str(i)
            print(row['prefix_2'], prefix)
            row = {"trace_id": case_id, "task": row[prefix], "start_timestamp": row['start:timestamp_'+str(i)], "time_timestamp": row['time:timestamp_'+str(i)]}
            df_new = pd.concat([df_new, pd.DataFrame([row])], ignore_index=True)
    return df_new

original_train = pd.read_csv("/Users/francescameneghello/Documents/GitHub/nirdizati-light/experiments/new_logs/" +
                             "Productions_train_df_cf_aug_0.1_pref_len_20.csv")
transform_df(original_train, 20)

Case178
Round Grinding prefix_1


/var/folders/w4/c4bpfq1j4l94r7q69c434q0c0000gn/T/ipykernel_17124/1226155789.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_new, pd.DataFrame([row])], ignore_index=True)


KeyError: 'prefix_2'

In [27]:
original_train = pd.read_csv("/Users/francescameneghello/Documents/GitHub/nirdizati-light/experiments/new_logs/" +
                             "Productions_train_df_cf_aug_0.1_pref_len_20.csv")
original_train = prefix_to_log(original_train, 9)

original_train = pm4py.convert_to_dataframe(original_train)

#original_train[event_log_ids.start_time] = pd.to_datetime(original_train[event_log_ids.start_time], utc=True)
#original_train[event_log_ids.end_time] = pd.to_datetime(original_train[event_log_ids.end_time], utc=True)

event_log_ids = EventLogIDs(  # These values are stored in DEFAULT_CSV_IDS
        case="caseid",
        activity="task",
        start_time="start:timestamp",
        end_time="time:timestamp"
    )

path = '/Users/francescameneghello/Documents/GitHub/nirdizati-light/experiments/new_logs/ConsultaDataMining201618/ConsultaDataMining201618_train_df_cf_simulated_aug_*'
#all_file = glob.glob(path_dict[t][0])

distance = cycle_time_distribution_distance(
                            original_train, event_log_ids,  # First event log and its column id mappings
                            original_train, event_log_ids,  # Second event log and its column id mappings
                            bin_size=pd.Timedelta(hours=1)  # Bins of 1 hour
                        )

ValueError: dictionary update sequence element #0 has length 1; 2 is required